## Predicting Property Maintenance Fines

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

Each row in train.csv and test.csv corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Goal

Predictions will be given as the probability that the corresponding blight ticket will be paid on time.

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

def blight_model():
    
    data = pd.read_csv('train.csv', encoding = 'latin1', low_memory = False).set_index('ticket_id')
    data['fine_amount'] = data['fine_amount'].fillna(data['fine_amount'].mean())
    data['compliance'] = data['compliance'].fillna(False)
    
    address = pd.read_csv('addresses.csv', encoding = 'latin1').sort_values('address')
    latlon = pd.read_csv('latlons.csv', encoding = 'latin1').sort_values('address')
    address = address.drop_duplicates(subset=['address'])
    loc = address.merge(latlon).set_index('ticket_id')
    loc['number'] = loc['address'].str.split(n=1).str[0].astype('int64') > 0
    loc = loc[loc['number'] == True].drop('number', axis = 1)
    
    data = data.dropna(axis=1)
    data = data.join(loc, how = 'left').dropna(axis=0)
    features = list(data.dtypes[data.dtypes != object].reset_index().iloc[:,0])
    features = features[1:7] + features[10:]
    
    X_train = data[features].drop(['compliance'], axis = 1)
    y_train = data['compliance']
    
    X_test = pd.read_csv('test.csv', encoding = 'latin1').set_index('ticket_id')
    X_test = X_test.join(loc, how = 'left')
    
    test_features = features[0:6] + features[7:]
    X_test = X_test[test_features].fillna(0)

    #X_train, X_test, y_train, y_test = train_test_split(X_, y_, random_state=0)
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    lrRidge = Ridge().fit(X_train, y_train)
    
    clf = GradientBoostingClassifier(learning_rate = 0.01).fit(X_train, y_train)
#     clf = LogisticRegression(C=100).fit(X_train, y_train)
#     clf = SVC(kernel = 'linear', C=100).fit(X_train, y_train)
    y_prob = clf.predict_proba(X_test)
    
    answer = pd.Series(data = y_prob[:,1], index = X_test.index)
    
    return answer #roc_auc_score(y_test, y_prob[:,1]) #Your answer here
blight_model()

ticket_id
284932    0.2
285362    0.0
285361    0.3
285338    0.0
285346    0.3
285345    0.5
285347    0.4
285342    0.5
285530    0.6
284989    0.0
285344    0.4
285343    0.0
285340    0.6
285341    0.4
285349    0.0
285348    0.5
284991    0.3
285532    0.3
285406    0.3
285001    0.3
285006    0.6
285405    0.6
285337    0.3
285496    0.4
285497    0.5
285378    0.0
285589    0.3
285585    0.5
285501    0.3
285581    0.6
         ... 
376367    0.0
376366    0.4
376362    0.4
376363    0.5
376365    0.3
376364    0.4
376228    0.6
376265    0.0
376286    0.4
376320    0.0
376314    0.0
376327    0.7
376385    0.7
376435    0.6
376370    0.7
376434    0.1
376459    0.0
376478    0.4
376473    0.1
376484    0.3
376482    0.3
376480    0.3
376479    0.3
376481    0.0
376483    0.4
376496    0.6
376497    0.0
376499    0.0
376500    0.0
369851    0.0
dtype: float64

In [26]:
blight_model()

(61001,)